1. Считать таблицу из файла Flags.csv. По описаниям из файла FlagsDescript.txt 
   определить тип шкалы для каждого признака (в классификации из соответствующей лекции).
2. Создать функцию, вычисляющую расстояние между объектами по формулам из лекции с учетом типа шкал.
3. Вычислить матрицу попарных расстояний для таблицы.
4. Найти объект, суммарное расстояние до которого от всех остальных объектов максимально.
5. Создать функцию, вычисляющую расстояние между порядковыми признаками по формуле из лекции.
6. Вычислить расстояние между любыми двумя порядковыми признаками из таблицы.

1.Страна	
2.Географическая зона 1=NE, 2=SE, 3=SW, 4=NW	
3.Площадь	
4.Население	
5.Континент (в порядке убывания площади) 1=Asia,2=Africa,3=N.America,4=S.America,5=Europe,6=Oceania 	
6.Континент (в порядке убывания населения) 1=Asia,2=Africa,3=Europe,4=N.America,5=S.America,6=Oceania 	
7.Язык	1=English, 2=Spanish, 3=French, 4=German, 5=Slavic, 6=Other Indo-European, 7=Chinese, 8=Arabic, 9=Japanese/Turkish/Finnish/Magyar, 10=Others 	
8.Религия 0=Catholic, 1=Other Christian, 2=Muslim, 3=Buddhist, 4=Hindu, 5=Ethnic, 6=Marxist, 7=Others 	
9.Число вертикальных полос в флаге  	
10.Число горизонтальных полос в флаге	
11.Число различных цветов в флаге	
12.Красный (наличие цвета в флаге)	
13.Зеленый (наличие цвета в флаге)	
14.Синий (наличие цвета в флаге)	 
15.Золотой (наличие цвета в флаге)	
16.Белый (наличие цвета в флаге)	
17.Черный (наличие цвета в флаге)	
18.Оранжевый (наличие цвета в флаге)	
19.Преобладающий цвет													

In [197]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter

# Задание 1.

| N | Feature   | Type  |
|---|---|---|
|  1 | Страна  |   |
|  2 | Географическая зона  | НАИМЕНОВАНИЙ  |
|  3 | Площадь  | ОТНОШЕНИЯ  |
|  4 | Население  | ОТНОШЕНИЯ  |
|  5 | Континент (в порядке убывания площади)  | ПОРЯДКА  |
|  6 | Континент (в порядке убывания населения)  | ПОРЯДКА  |
|  7 | Язык  | НАИМЕНОВАНИЙ  |
|  8 | Религия  | НАИМЕНОВАНИЙ  |
|  9 | Число вертикальных полос в флаге  | АБСОЛЮТНАЯ  |
|  10 | Число горизонтальных полос в флаге  | АБСОЛЮТНАЯ  |
|  11 | Число различных цветов в флаге  | АБСОЛЮТНАЯ  |
|  12 | Красный  | НАИМЕНОВАНИЙ  |
|  13 | Зеленый  | НАИМЕНОВАНИЙ  |
|  14 | Синий  | НАИМЕНОВАНИЙ  |
|  15 | Золотой  | НАИМЕНОВАНИЙ  |
|  16 | Белый  | НАИМЕНОВАНИЙ  |
|  17 | Черный  | НАИМЕНОВАНИЙ  |
|  18 | Оранжевый  | НАИМЕНОВАНИЙ  |
|  19 | Преобладающий цвет  | НАИМЕНОВАНИЙ  |

In [198]:
names = ['country', 'geographic_area', 'area', 'population', 'continent_desc_area', 'continent_desc_pop', 'language', 'religion',
        'vert_count', 'horiz_count', 'colours_count', 'red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'prevailing']
name_features = ['geographic_area', 'language', 'religion', 'red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'prevailing']
order_features = ['continent_desc_area', 'continent_desc_pop']
rel_features = ['area', 'population']
abs_features = ['vert_count', 'horiz_count', 'colours_count']

In [199]:
data = pd.read_csv('Flags.csv', sep=';', names=names)

In [200]:
data.head(10)

,country,geographic_area,area,population,continent_desc_area,continent_desc_pop,language,religion,vert_count,horiz_count,colours_count,red,green,blue,gold,white,black,orange,prevailing
0,Afghanistan,1,648,16,1,1,10,2,0,3,5,1,1,0,1,1,1,0,green
1,Albania,1,29,3,5,3,6,6,0,0,3,1,0,0,1,0,1,0,red
2,Algeria,1,2388,20,2,2,8,2,2,0,3,1,1,0,0,1,0,0,green
3,American-Samoa,3,0,0,6,6,1,1,0,0,5,1,0,1,1,1,0,1,blue
4,Andorra,1,0,0,5,3,6,0,3,0,3,1,0,1,1,0,0,0,gold
5,Angola,2,1247,7,2,2,10,5,0,2,3,1,0,0,1,0,1,0,red
6,Anguilla,4,0,0,3,4,1,1,0,1,3,0,0,1,0,1,0,1,white
7,Antigua-Barbuda,4,0,0,3,4,1,1,0,1,5,1,0,1,1,1,1,0,red
8,Argentina,3,2777,28,4,5,2,0,0,3,2,0,0,1,0,1,0,0,blue
9,Argentine,3,2777,28,4,5,2,0,0,3,3,0,0,1,1,1,0,0,blue


# Задание 2.

In [201]:
def order_dist(a, b, c_name):
    t_a = len(data[data[c_name] == a])
    h_a = 0
    x_a = 0
    for i in range(a - 1):
        h_a += data[c_name].value_counts().sort_index().iloc[i]
    for j in range(1, t_a + 1):
        x_a += j + h_a
    x_a /= t_a

    t_b = len(data[data[c_name] == b])
    h_b = 0
    x_b = 0
    for i in range(b - 1):
        h_b += data[c_name].value_counts().sort_index().iloc[i]
    for j in range(1, t_b + 1):
        x_b += j + h_b
    x_b /= t_b
    res = abs(x_a - x_b) / data[c_name].count()
    return res


def strong_dist(a, b, c_name):
    max_x = data[c_name].max()
    min_x = data[c_name].min()
    res = abs(a - b) / (max_x - min_x)
    return res


def name_dist(a, b, c_name):
    res = 0
    if a != b:
        a_l = len(data[data[c_name] == a])
        b_l = len(data[data[c_name] == b])
        res = (a_l + b_l) / len(data[c_name])
    return res


def distance(a, b):
    result = 0
    for feature in name_features:
        result += name_dist(a[feature], b[feature], feature) ** 2
    for feature in order_features:
        result += order_dist(a[feature], b[feature], feature) ** 2
    for feature in rel_features + abs_features:
        result += strong_dist(a[feature], b[feature], feature) ** 2
    result /= len(name_features + order_features + rel_features + abs_features)
    result = np.sqrt(result)
    return result

# Задание 3.

In [202]:
matr = np.zeros((data.shape[0], data.shape[0]))
for i in range(data.shape[0]):
    for j in range(data.shape[0]):
        matr[i][j] = distance(data.iloc[i], data.iloc[j])
matr

array([[0.        , 0.43054611, 0.37385927, ..., 0.45121343, 0.47284783,
        0.19435242],
       [0.43054611, 0.        , 0.51926477, ..., 0.48043228, 0.50075401,
        0.43868739],
       [0.37385927, 0.51926477, 0.        , ..., 0.45857163, 0.45131027,
        0.41500003],
       ...,
       [0.45121343, 0.48043228, 0.45857163, ..., 0.        , 0.36213276,
        0.42631029],
       [0.47284783, 0.50075401, 0.45131027, ..., 0.36213276, 0.        ,
        0.44911476],
       [0.19435242, 0.43868739, 0.41500003, ..., 0.42631029, 0.44911476,
        0.        ]])

# Задание 4.

In [203]:
dists = matr.sum(axis=1)
dists.argmax()

157

In [204]:
data.iloc[157]

country                Sri-Lanka
geographic_area                1
area                          66
population                    15
continent_desc_area            1
continent_desc_pop             1
language                      10
religion                       3
vert_count                     2
horiz_count                    0
colours_count                  4
red                            0
green                          1
blue                           0
gold                           1
white                          0
black                          0
orange                         1
prevailing                  gold
Name: 157, dtype: object

# Задание 5.

In [205]:
def o_f_dist(a_name, b_name):
    combs = list(itertools.combinations(data['country'], 2))
    c_num = len(combs)
    res = 0
    for comb in combs:
        a_i = data[data.country == comb[0]][a_name].item()
        a_l = data[data.country == comb[1]][a_name].item()
        b_i = data[data.country == comb[0]][b_name].item()
        b_l = data[data.country == comb[1]][b_name].item()
        if a_i > a_l and b_i > b_l \
        or a_i < a_l and b_i < b_l \
        or a_i == a_l and b_i == b_l:
            pass
        elif a_i > a_l and b_i < b_l \
        or a_i < a_l and b_i > b_l:
            res += 1
        elif a_i != a_l and b_i == b_l \
        or a_i == a_l and b_i != b_l:
            res += 0.5
    res /= c_num
    return res

# Задание 6.

In [206]:
o_f_dist('continent_desc_area', 'continent_desc_pop')

0.08973879600448693